In [1]:
from mosqlimate_assistant.assistant import AssistantGemini
from mosqlimate_assistant.api_consumer import generate_api_url
from mosqlimate_assistant.faiss_db import get_relevant_sample_asks
from pprint import pprint

### 1. Pergunta do Usuário

In [2]:
question = "Quais são os dados da dengue no Acre no primeiro trimestre de 2019?"

In [3]:
assistente = AssistantGemini()

### 2. Verificar se a pergunta é relevante
##### Aqui usamos o ollama e um banco de dados fixo

In [4]:
relevant_questions, scores = get_relevant_sample_asks(question)

In [5]:
print("Perguntas relevantes:")
for relevant_question, score in zip(relevant_questions, scores):
    print(f"- score: {score:.2f} | {relevant_question['question']}")


Perguntas relevantes:
- score: 0.78 | Quero registros de dengue em João Pessoa, PB no segundo semestre de 2019
- score: 0.78 | Quero dados de denguede todo o Brasil em 2020
- score: 0.78 | Quero registros de dengue em Manaus, AM em 2019


### 3. Fazer a pergunta ao modelo com exemplos relevantes
##### Obs: O pipeline já está configurado para adicionar exemplos relevantes

In [6]:
model_answer = assistente.query_llm(
    question,
    relevant_questions,
    False,
    "."
)

In [7]:
pprint(model_answer)

{'city': None,
 'disease': 'dengue',
 'end': '2019-03-31',
 'key': None,
 'start': '2019-01-01',
 'table': 'infodengue',
 'uf': 'AC',
 'year': None}


### 4. Filtrar a resposta

In [8]:
filtered_answer = assistente.get_table_filters(model_answer)

In [9]:
pprint(filtered_answer)

InfodengueFilters(table='infodengue', disease='dengue', start='2019-01-01', end='2019-03-31', uf='AC', city=None)


### 5. Convertendo a resposta

In [10]:
api_url = generate_api_url(filtered_answer)
python_base_code = assistente.generate_mosqlient_infodengue_code(filtered_answer)

In [11]:
api_url

'https://api.mosqlimate.org/api/datastore/infodengue?page=1&per_page=100&disease=dengue&start=2019-01-01&end=2019-03-31&uf=AC'

In [12]:
print(python_base_code)

from mosqlient import get_infodengue

# return a pd.DataFrame with the data
df = get_infodengue(start_date='2019-01-01',end_date='2019-03-31',disease='dengue',uf='AC')
